# Get trending topics from Google Search and YouTube

### env config

In [1]:
import os
import sys
import ipykernel

# from dotenv import load_dotenv
# load_dotenv()  # this loads the .env script for use below

PROJECT_ID = os.getenv("PROJECT_ID")
LOCATION = os.getenv("LOCATION")
PREFIX = os.getenv("PREFIX")

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"LOCATION: {LOCATION}")
print(f"PREFIX: {PREFIX}")

PROJECT_ID: hybrid-vertex
LOCATION: us-central1
PREFIX: zghost_v1


### imports

In [2]:
import warnings
warnings.filterwarnings('ignore')

from google.cloud import bigquery
import pandas as pd
import numpy as np
# from datetime import datetime, timedelta

bq = bigquery.Client(project = PROJECT_ID)

## Search Trends

see overview and samples for the [Search Trends](https://pantheon.corp.google.com/marketplace/product/bigquery-public-datasets/google-search-trends?_ga=2.261190030.2019434361.1656948847-1975246695.1656948843&e=13802955&mods=-ai_platform_fake_service,-ai_platform_staging_service&inv=1&invt=AbtEjw&project=hybrid-vertex) (BigQuery public dataset)

> see code for [generating time intervals](https://source.corp.google.com/piper///depot/google3/experimental/marketing/trendcrawler/trend_crawler/src/data/transformers.py;ws=jwortz%2F310)

In [3]:
TARGET_DATE = "03/25/2025"

In [4]:
query = f"""
    SELECT
        term,
        refresh_date,
        ARRAY_AGG(STRUCT(rank,week) ORDER BY week DESC LIMIT 1) x
    FROM 
        `bigquery-public-data.google_trends.top_terms`
    WHERE
        refresh_date = PARSE_DATE('%m/%d/%Y',  '{TARGET_DATE}')
    GROUP BY    
        term, refresh_date
    ORDER BY
        (SELECT rank FROM UNNEST(x))
"""
print(query)


    SELECT
        term,
        refresh_date,
        ARRAY_AGG(STRUCT(rank,week) ORDER BY week DESC LIMIT 1) x
    FROM 
        `bigquery-public-data.google_trends.top_terms`
    WHERE
        refresh_date = PARSE_DATE('%m/%d/%Y',  '03/25/2025')
    GROUP BY    
        term, refresh_date
    ORDER BY
        (SELECT rank FROM UNNEST(x))



In [5]:
target_trends_df = bq.query(query).to_dataframe()
target_trends_df.head(10)

,term,refresh_date,x
0,bolivia vs uruguay,2025-03-25,"[{'rank': 1, 'week': 2025-03-23 00:00:00}]"
1,bolivia - uruguay,2025-03-25,"[{'rank': 2, 'week': 2025-03-23 00:00:00}]"
2,jasmine crockett,2025-03-25,"[{'rank': 3, 'week': 2025-03-23 00:00:00}]"
3,troy taylor,2025-03-25,"[{'rank': 4, 'week': 2025-03-23 00:00:00}]"
4,outside lands 2025,2025-03-25,"[{'rank': 5, 'week': 2025-03-23 00:00:00}]"
5,aaliyah chavez,2025-03-25,"[{'rank': 6, 'week': 2025-03-23 00:00:00}]"
6,is chipotle closing,2025-03-25,"[{'rank': 7, 'week': 2025-03-23 00:00:00}]"
7,erik per sullivan,2025-03-25,"[{'rank': 8, 'week': 2025-03-23 00:00:00}]"
8,nigeria vs zimbabwe,2025-03-25,"[{'rank': 9, 'week': 2025-03-23 00:00:00}]"
9,gme stock,2025-03-25,"[{'rank': 10, 'week': 2025-03-23 00:00:00}]"


In [6]:
sample_term = target_trends_df['term'].iloc[0]
print(sample_term)

bolivia vs uruguay


# YouTube Trends

references
* video resource [JSON structure](https://developers.google.com/youtube/v3/docs/videos#resource-representation)

### config

In [7]:
import pandas as pd
from IPython.display import HTML

from google.cloud import secretmanager
import googleapiclient.discovery
import googleapiclient.errors

# secret manager client
sm_client = secretmanager.SecretManagerServiceClient()

API_SERVICE_NAME = "youtube"
API_VERSION = "v3"

_SECRET_ID = 'projects/934903580331/secrets/yt-data-api'
_SECRET_VERSION = '{}/versions/1'.format(_SECRET_ID)
_SECRET_NAME = sm_client.secret_path(PROJECT_ID, _SECRET_ID)
print(f"_SECRET_NAME: {_SECRET_NAME}\n")

_SECRET_NAME: projects/hybrid-vertex/secrets/projects/934903580331/secrets/yt-data-api



### setup yt discovery client

> see [src](https://github.com/googleapis/google-api-python-client/blob/main/googleapiclient/discovery.py)

In [8]:
response = sm_client.access_secret_version(request={"name": _SECRET_VERSION})
# print(f"response: {response}")

YOUTUBE_DATA_API_KEY = response.payload.data.decode("UTF-8")
# print(f"YOUTUBE_DATA_API_KEY: {YOUTUBE_DATA_API_KEY}")

youtube = googleapiclient.discovery.build(
    serviceName=API_SERVICE_NAME, 
    version=API_VERSION, 
    developerKey=YOUTUBE_DATA_API_KEY
)

youtube

In [9]:
TARGET_QUERY = ""
CHANNEL_ID = "UCvW9uSNy6Lytcnib1CdXrow"

NUM_RESULTS = 10

# duration (minutes): ['any', 'long', 'medium', 'short']
# short: (-inf, 4)
# medium: [4, 20]
# long: (20, inf)
VIDEO_DURATION = "short"

# order resources in API response
ORDER_CRITERIA = "viewCount"  # ['date', 'rating', 'relevance', 'title', 'viewCount']

MAX_DAYS_AGO = 10
PUBLISHED_AFTER_TIMESTAMP = (
    (pd.Timestamp.now() - pd.DateOffset(days=MAX_DAYS_AGO))
    .tz_localize("UTC")
    .isoformat()
)

print(f"search_query: {TARGET_QUERY}")
print(f"channelId: {CHANNEL_ID}")
print(f"video_duration_type: {VIDEO_DURATION}")
print(f"published_after: {PUBLISHED_AFTER_TIMESTAMP}")
print(f"order_criteria: {ORDER_CRITERIA}")
print(f"num_results: {NUM_RESULTS}")

search_query: 
channelId: UCvW9uSNy6Lytcnib1CdXrow
video_duration_type: short
published_after: 2025-03-16T21:50:15.591400+00:00
order_criteria: viewCount
num_results: 10


create dataframe from response dictionary

```python
data = {
    'row1': {'col1': 1, 'col2': 2},
    'row2': {'col1': 3, 'col2': 4},
    'row3': {'col1': 5, 'col2': 6}
}

```

In [ ]:
# AS_OF_DATE = "20240103"
# default_date = datetime.strptime(AS_OF_DATE, "%Y%m%d")
# selected_date_str = default_date.strftime("%Y%m%d")
# print("As of date: ", default_date)

TODO
* see options for `part` parameter:
  * [contentDetails](https://developers.google.com/youtube/v3/docs/videos#contentDetails)
  * [statisitcs](https://developers.google.com/youtube/v3/docs/videos#statistics)
  * [topicDetails](https://developers.google.com/youtube/v3/docs/videos#topicDetails)

In [13]:
# Using Search:list - https://developers.google.com/youtube/v3/docs/search/list
yt_data_api_request = youtube.search().list(
    part="id,snippet", # contentDetails,statistics,topicDetails
    type="video",
    q=TARGET_QUERY,
    videoDuration=VIDEO_DURATION,
    maxResults=NUM_RESULTS,
    publishedAfter=PUBLISHED_AFTER_TIMESTAMP,
    channelId=CHANNEL_ID,
    order=ORDER_CRITERIA,
)
yt_data_api_response = yt_data_api_request.execute()

# prepare results dataframe
data_dict = {}
i=1

for video in yt_data_api_response['items']:
    row_name = f"row{i}"
    data_dict.update(
        {
            row_name: {
                'publishedAt': video['snippet']['publishedAt'],
                'videoId': video['id']['videoId'], 
                'videoTitle': video['snippet']['title'],
                'videoDescription': video['snippet']['description'],
                'videoThumbnail': video['snippet']['thumbnails']['high']['url'],
                'videoURL': f"https://www.youtube.com/watch?v={video['id']['videoId']}",
                'channelTitle': video['snippet']['channelTitle'],
                'channelId': video['snippet']['channelId'],
            }
        }
    )
    i+=1

yt_df = pd.DataFrame.from_dict(data_dict, orient='index')
yt_df = yt_df.reset_index(drop=True)

HTML(yt_df.to_html(render_links=True, escape=False))

,publishedAt,videoId,videoTitle,videoDescription,videoThumbnail,videoURL,channelTitle,channelId
0,2025-03-18T16:00:06Z,o8iYmeqXU20,Gettin' Cooked With Ricky - Sneak Preview!,"Where the best setting is BAKED... Gettin' Cooked With Ricky launches on SwearNet Friday, March 28! #trailerparkboys ...",https://i.ytimg.com/vi/o8iYmeqXU20/hqdefault.jpg,https://www.youtube.com/watch?v=o8iYmeqXU20,Trailer Park Boys,UCvW9uSNy6Lytcnib1CdXrow
1,2025-03-25T01:15:01Z,E55Pw55MvGA,Park After Dark S6E44 - Gimme The F**king Liquor!,Now streaming at https://bit.ly/PAD6-ep44 and the TPB SwearNet app: It's officially BOOZE MONDAY! The Boys are back at a new ...,https://i.ytimg.com/vi/E55Pw55MvGA/hqdefault.jpg,https://www.youtube.com/watch?v=E55Pw55MvGA,Trailer Park Boys,UCvW9uSNy6Lytcnib1CdXrow
2,2025-03-17T12:15:06Z,tmGzwM3pDls,Don't mess with the Green Bastard ✳️ #trailerparkboys #greenbastard #jimlahey,Green Bastard fights Ricky and Jim Lahey saves the day... is this some kinda parallel fackin' universe?!?,https://i.ytimg.com/vi/tmGzwM3pDls/hqdefault.jpg,https://www.youtube.com/watch?v=tmGzwM3pDls,Trailer Park Boys,UCvW9uSNy6Lytcnib1CdXrow
3,2025-03-17T16:15:04Z,h6GQonIkFL4,"Gettin' Cooked With Ricky - Watch the trailer on Tues, March 18!","IT'S COMIN'... Check back TOMORROW, March 18 at 12pm ET for a sneak peek of Ricky (and Randy's!) new show ...",https://i.ytimg.com/vi/h6GQonIkFL4/hqdefault.jpg,https://www.youtube.com/watch?v=h6GQonIkFL4,Trailer Park Boys,UCvW9uSNy6Lytcnib1CdXrow
4,2025-03-26T17:18:20Z,xUlsza9ciu0,Friday. 9PM ET. SwearNet. Are you fuckin' ready to GET COOKED? 🍗💨🍒 # #trailerparkboys #cookingshow,Where the best setting is BAKED... Gettin' Cooked With Ricky launches on https://SwearNet.com and the TPB SwearNet app on ...,https://i.ytimg.com/vi/xUlsza9ciu0/hqdefault.jpg,https://www.youtube.com/watch?v=xUlsza9ciu0,Trailer Park Boys,UCvW9uSNy6Lytcnib1CdXrow
5,2025-03-25T16:15:00Z,6ypNwuYfUs4,Standing On The Shoulders Of Kitties DVD & Blu-Ray! #BubblesAndTheShitrockers #TrailerParkBoys,HOOOOEEYYY! Standing On The Shoulders Of Kitties is now available on DVD and Blu-Ray (includes extras and bloopners!),https://i.ytimg.com/vi/6ypNwuYfUs4/hqdefault.jpg,https://www.youtube.com/watch?v=6ypNwuYfUs4,Trailer Park Boys,UCvW9uSNy6Lytcnib1CdXrow


In [14]:
yt_data_api_response['items'][0]

{'kind': 'youtube#searchResult',
 'etag': 'Q48eg7EnewvJJe_vEz33obHOivU',
 'id': {'kind': 'youtube#video', 'videoId': 'o8iYmeqXU20'},
 'snippet': {'publishedAt': '2025-03-18T16:00:06Z',
  'channelId': 'UCvW9uSNy6Lytcnib1CdXrow',
  'title': 'Gettin&#39; Cooked With Ricky - Sneak Preview!',
  'description': "Where the best setting is BAKED... Gettin' Cooked With Ricky launches on SwearNet Friday, March 28! #trailerparkboys ...",
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/o8iYmeqXU20/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {'url': 'https://i.ytimg.com/vi/o8iYmeqXU20/mqdefault.jpg',
    'width': 320,
    'height': 180},
   'high': {'url': 'https://i.ytimg.com/vi/o8iYmeqXU20/hqdefault.jpg',
    'width': 480,
    'height': 360}},
  'channelTitle': 'Trailer Park Boys',
  'liveBroadcastContent': 'none',
  'publishTime': '2025-03-18T16:00:06Z'}}

## Load to respond dataframe to BigQuery

In [15]:
BQ_DATASET = 'youtube_video_analysis' # os.getenv("BQ_DATASET")
TRENDING_YT_RESPONSE_TABLE = "yt_trends_responses_v2"

In [16]:
YOUTUBE_API_LOAD_JOB = bq.load_table_from_dataframe(
    yt_df,
    f"{BQ_DATASET}.{TRENDING_YT_RESPONSE_TABLE}",
    # job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"),
)

# Wait for the load job to complete
YOUTUBE_API_LOAD_JOB.result()

LoadJob<project=hybrid-vertex, location=us-central1, id=61da4191-80ee-46c0-a2be-36f9ee312f65>